In [25]:
import heapq
import folium
from folium.plugins import HeatMap
import branca.colormap as cm

class NetworkNode:
    def __init__(self, ID, latitude, longitude):
        self.ID = ID
        self.latitude = latitude
        self.longitude = longitude

    def __repr__(self):
        return f"{self.__class__.__name__}({self.ID})"

    def __hash__(self):
        return hash(self.ID)

    def __eq__(self, other):
        return (self.ID == other.ID) and (type(self) is type(other))

class FogCloudNode(NetworkNode):
    def __init__(self, ID, longitude, latitude, processing_capacity, memory_capacity, cost, model):
        super().__init__(ID, latitude, longitude)
        self.model = model
        self.processing_capacity = processing_capacity
        self.memory_capacity = memory_capacity
        self.cost = cost
        self.requisicoes = 0

class FogNode(FogCloudNode):
    pass

class CloudNode(FogCloudNode):
    pass

class Sensor(NetworkNode):
    def __init__(self, ID, longitude, latitude, services):
        super().__init__(ID, latitude, longitude)
        self.services = services
        
class Aresta:
    def __init__(self, u, v, largura_banda, custo, tempo):
        self.u = u
        self.v = v
        self.largura_banda = largura_banda
        self.custo = custo
        self.tempo = tempo
        self.quantidade_uso = 0
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.u}, {self.v})"
        
class Grafo:
    def __init__(self):
        self.adj = {}
        self.adj_ = {}
        self.n_vertices = 0
        self.n_arestas = 0

    def add_vertice(self, noh):
        if noh not in self.adj:
            self.adj[noh] = []
            self.n_vertices += 1

    def add_aresta(self, u, v, largura_banda, custo, dist_H):
        self.add_vertice(u)
        self.add_vertice(v)
        tempo = 0.001 + float(dist_H)/(2*10**8)
        self.adj[u].append((v, largura_banda, custo, tempo))
        aresta = Aresta(u, v, largura_banda, custo, tempo)
        aresta.largura_banda = float(largura_banda)*10**9
        self.adj_[f"({u},{v})"] = aresta
        self.n_arestas += 1

    def copiar(self):
        novo = Grafo()
        novo.n_arestas =  self.n_arestas
        for vertice, arestas in self.adj.items():
            novo.add_vertice(vertice)
            for v, largura_banda, custo, tempo in arestas:
                novo.adj[vertice].append((v, largura_banda, custo, tempo))
        return novo


def floyd_warshall(grafo):
    dist = {u: {v: float('inf') for v in grafo.adj.keys()} for u in grafo.adj.keys()}
    prev = {u: {v: None for v in grafo.adj.keys()} for u in grafo.adj.keys()}

    for u in grafo.adj.keys():
        dist[u][u] = 0
        for v, _, _, t in grafo.adj[u]:
            dist[u][v] = t
            prev[u][v] = u

    for k in grafo.adj.keys():
        for i in grafo.adj.keys():
            for j in grafo.adj.keys():
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    prev[i][j] = prev[k][j]
    return dist, prev


def reconstruir_caminho(prev, origem, destino):
    caminho = []
    atual = destino
    while atual is not None:
        caminho.append(atual)
        atual = prev[origem][atual]
    caminho.reverse()
    return caminho


def processar_caminho(grafo, caminho, requisicao, instante):
    arcos = []
    band_tot = 0
    custo = 0
    selecionado = None
    for i in range(len(caminho)-1):
        vertice_atual = caminho[i]
        vizinhos = grafo.adj[vertice_atual]
        for vizinho in vizinhos:
            if(vizinho[0] == caminho[i+1]) and (requisicao[2] <= grafo.adj_[f"({vertice_atual},{vizinho[0]})"].largura_banda):
                selecionado = vizinho[0]
                arcos.append(f"({vertice_atual},{selecionado})")
                grafo.adj_[f"({vertice_atual},{selecionado})"].largura_banda -= requisicao[2]
                # Em qual instante adicionar a aresta?
                temporal_arestas[instante].append((grafo.adj_[f"({vertice_atual},{selecionado})"], requisicao[2]))
                band_tot += float(vizinho[1])
                custo += float(vizinho[1])*float(vizinho[2])
                break
        if selecionado is None:
            return selecionado, False, arcos, band_tot, custo
        if(requisicao[0] <= selecionado.processing_capacity):
            if(requisicao[1] <= selecionado.memory_capacity):
                selecionado.processing_capacity -= requisicao[0]
                selecionado.memory_capacity -= requisicao[1]
                selecionado.requisicoes += 1
                return selecionado, True, arcos, band_tot, custo
    return selecionado, False, arcos, band_tot, custo

# Demais leitura do arquivo e parte principal do programa permanece a mesma...
# Substituir o trecho que chama dijkstra por:
#
# tempo_dict = dist[sensor]
# candidatos = sorted([(v, t) for v, t in tempo_dict.items() if isinstance(v, FogCloudNode)], key=lambda x: x[1])
# for destino, _ in candidatos:
#     caminho = reconstruir_caminho(prev, sensor, destino)
#     selecionado, processou, arcos, band, c = processar_caminho(...)
#     if processou: break 
        

# Próximos passos - leitura da instância na main
# Leitura e armazenamentos dos dados até "#end_services"
    # instance_info -> cabeçalho (apenas algumas informações são importantes
    # sensors -> ler e armazenar numa lista de objetos Sensor()
    # reach_fog_nodes -> ler e armazenar num grafo as arestas Sensor-Fog
    # fog -> ler e armazenar numa lista de objetos FogNode() e arestas Fog-Fog
    # cloud -> ler e armazenar numa lista de objetos CloudNode() e arestas Fog-Cloud
    # service -> ler e armazenar num dicionário de services, com a chave sendo o nome do serviço
##
# Começa a leitura das requests
    # Para cada time_instant
        # Armazena o valor do time instant numa variável i
        # Verifica na ED temporal se naquele i precisa de alteração no grafo (retorno de disponibilidade de capacidade)
        # Se sim, realiza todas as alterações necessárias
        # Roda o Dijkstra
        # Começa a percorrer as requisições do tipo "sensor ID | service index | type of service | request_lifetime"
        # Chama a função processar_caminho() para essa requisição
        # Se True
            # ED Temporal: adiciona para o instante i+request_lifetime+1 o valor (FogCloudNode, services[type_of_service][0], services[type_of_service][1])
            # Numero de requisições aceitas: incrementado
            # Número de requisições do tipo "type of service" aceitas: incrementado
            # Dentro de processar_caminho()
                # Sinaliza o nó fog em que foi aceita a requisição
                # Conta a quantidade de arcos utilizados do sensor inicial até o nó fog em que foi processado
                # Soma a quantidade de largura de banda 
                # Soma o custo dos arcos com o custo do processamento
                # Retorna todos esse valores
# Após o "#end_requests", retorna todos os valores de dados de análise e gera-se o mapa de calor 

with open("9.txt", "r", encoding="utf-8") as f:
    grafo = Grafo()
    for i in range(7):
        f.readline()
    linha = f.readline().strip()
    num_sensores = int(linha.split()[1])
    linha = f.readline().strip()
    num_fog = int(linha.split()[1])
    linha = f.readline().strip()
    num_cloud = int(linha.split()[1])
    linha = f.readline().strip()
    num_servicos = int(linha.split()[1])
    f.readline()
    f.readline()
    sensores = []
    for i in range(num_sensores):
        linha = f.readline().strip().split()
        # Sensores -> index | longitude | latitude | services
        sensor = Sensor(linha[0], linha[1], linha[2], linha[3::])
        sensores.append(sensor)
        grafo.add_vertice(sensor)
    f.readline()
    f.readline()
    fogs = []
    for i in range(num_fog):
        linha = f.readline().strip().split()
        # Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
        fog = FogNode(linha[0], float(linha[1]), float(linha[2]), float(linha[3]), float(linha[4]), float(linha[5]), linha[6])
        fogs.append(fog)
        grafo.add_vertice(fog)
    num_fogfog = int(f.readline().strip())
    for i in range(num_fogfog):
        linha = f.readline().strip().split()
        # Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
        grafo.add_aresta(fogs[int(linha[1])], fogs[int(linha[2])], linha[3], linha[4], linha[5])
    f.readline()
    f.readline()
    linha = f.readline().strip()
    while linha != "#end_reach_fog_nodes":
        linha = linha.split()
        # Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
        grafo.add_aresta(sensores[int(linha[1])], fogs[int(linha[2])], linha[3], linha[4], linha[5])
        linha = f.readline().strip()
    f.readline()
    clouds = []
    for i in range(num_cloud):
        linha = f.readline().strip().split()
        # Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
        cloud = CloudNode(linha[0], float(linha[1]), float(linha[2]), float(linha[3]), float(linha[4]), float(linha[5]), linha[6])
        clouds.append(cloud)
    num_fogcloud = int(f.readline().strip())
    for i in range(num_fogcloud):
        linha = f.readline().strip().split()
        # Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
        grafo.add_aresta(fogs[int(linha[1])], clouds[int(linha[2])], linha[3], linha[4], linha[5])
    f.readline()
    f.readline()
    servicos = {}
    for i in range(num_servicos):
        linha =  f.readline().strip().split()
        # ID: [processing_demand, memory_demand, number_of_bits, lifetime]
        servicos[linha[0]] = [float(linha[1]), float(linha[2]), int(linha[3]), int(linha[4])]
    f.readline() # #end_services
    f.readline() # #begin_requests
    ## PARTE 2 - Leitura das requests
    temporal = {i: [] for i in range(1, 1001)}
    temporal_arestas = {i: [] for i in range(1, 1001)}
    linha = f.readline().strip()
    tot_req = 0
    quant_req = 0
    set_arcos = []
    quant_band = 0
    quant_custo = 0
    dist, prev = floyd_warshall(grafo)
    while linha != "#end_requests":
        instante = int(linha.split("_")[2])
        for (v, p, m) in temporal[instante]:
            v.processing_capacity += p
            v.memory_capacity += m
        for aresta, band in temporal_arestas[instante]:
            aresta.largura_banda += band
        linha =  f.readline().strip()
        while (linha.split("_")[0] != "##time") and (linha != "#end_requests"):
            linha = linha.split()
            # sensor ID | service index | type of service | request_lifetime
            tot_req += 1
            sensor = sensores[int(linha[0])]
            tempo_dict = dist[sensor]
            candidatos = sorted([(v, t) for v, t in tempo_dict.items() if isinstance(v, FogNode)], key=lambda x: x[1])
            for destino, _ in candidatos:
                caminho = reconstruir_caminho(prev, sensor, destino)
                selecionado, processou, arcos, band, c = processar_caminho(grafo, caminho, servicos[linha[2]], instante)
                if processou: 
                    quant_req +=1
                    set_arcos += arcos
                    for arco in arcos:
                        grafo.adj_[arco].quantidade_uso += 1 
                    quant_band += band
                    quant_custo += c
                    #tX: [(Nó, processing_demand, memory_demand)]
                    if (instante+int(linha[3])+1) <= 1000:
                        temporal[instante+int(linha[3])+1].append((selecionado, servicos[linha[2]][0], servicos[linha[2]][1]))
                    break 
            linha = f.readline().strip()
    set_arcos = set(set_arcos)
    quant_arcos = len(set_arcos)
print(f"Total de requisições: {tot_req}\n % de requisições processadas: {(quant_req/tot_req)*100.0}%\n % de arcos usados: {(quant_arcos/grafo.n_arestas)*100.0}%\n Largura de banda usada: {quant_band}\n Custo gasto: {quant_custo}\n")

# Inicializa o mapa centralizado na média das coordenadas
avg_lat = sum(node.latitude for node in fogs) / len(fogs)
avg_lon = sum(node.longitude for node in fogs) / len(fogs)
mapa = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

# Constrói os dados para o mapa de calor
heat_data = [
    [node.latitude, node.longitude, node.requisicoes]
    for node in fogs if node.requisicoes > 0
]

# Adiciona camada de calor
HeatMap(heat_data).add_to(mapa)


for arco in grafo.adj_.values():
    u = arco.u
    v = arco.v
    uso = arco.quantidade_uso
    if uso > 0 and not (isinstance(u, CloudNode) or isinstance(v, CloudNode)):
        folium.PolyLine(
            locations=[[float(u.latitude), float(u.longitude)], [float(v.latitude), float(v.longitude)]],
            color="orange",
            weight=3,
            opacity=0.8,
            tooltip=f"{u.ID} → {v.ID}: {uso} usos"
        ).add_to(mapa)
    
# Adiciona marcadores (opcional)
for node in fogs:
    folium.Marker(
        location=[node.latitude, node.longitude],
        popup=f"Fog {node.ID} - {node.requisicoes} reqs",
        icon=folium.Icon(color="blue" if node.requisicoes > 0 else "gray")
    ).add_to(mapa)
    
for node in sensores:
    folium.Marker(
        location=[node.latitude, node.longitude],
        popup=f"Sensor {node.ID}",
        icon=folium.Icon(color="red")
    ).add_to(mapa)

# Salvar como HTML interativo
mapa.save("mapa_calor.html")

Total de requisições: 22450
 % de requisições processadas: 100.0%
 % de arcos usados: 41.89723320158103%
 Largura de banda usada: 957.4999999998231
 Custo gasto: 43.493655999996754

